In [1]:
#info_theory = {"camb": {"extra_args": {"lens_potential_accuracy": 1}}}
info_theory = {"classy": {"extra_args": {"non linear": "halofit"}}}

modules = "/home/jesus/scratch/modules/"

n = 200

In [2]:
from __future__ import division, print_function, absolute_import
from collections import OrderedDict as odict

from cobaya.conventions import _likelihood, _theory, _timing, _path_install
from cobaya.tools import recursive_update
from cobaya.cosmo_input import create_input, planck_base_model
from cobaya.model import get_model

from test_cosmo_planck_2015 import params_lowTEB_highTTTEEE

info = create_input(planck_names=True, theory=list(info_theory)[0], **planck_base_model)
info = recursive_update(info, {_theory: info_theory})

likelihoods = [
    "planck_2015_lowl", "planck_2015_lowTEB",
    "planck_2015_plikHM_TT", "planck_2015_plikHM_TTTEEE",
    "planck_2015_plikHM_TT_unbinned", "planck_2015_plikHM_TTTEEE_unbinned",
    "planck_2015_lensing", "planck_2015_lensing_cmblikes",
    "bicep_keck_2014",
    "H0_riess2018a", "H0_riess2018b",
    "sdss_dr12_consensus_bao", "sdss_dr12_consensus_full_shape", "sdss_dr12_consensus_final",
    "sdss_dr7_mgs", "sixdf_2011_bao",
    "sn_jla", "sn_jla_lite", "sn_pantheon"    
]

# NB: It's possible that DES makes everything else slower, since it adds many more
# redshifts to the computation, so there are many more redshifts to select the
# required e.g. H(z) from, making e.g. SN and BAO likes slower (longer theory code request)

info[_likelihood] = odict([[l,None] for l in likelihoods])

# No fsigma8 in classy!
if list(info_theory)[0] == "classy":
    info[_likelihood].pop("sdss_dr12_consensus_final")
    info[_likelihood].pop("sdss_dr12_consensus_full_shape")

info[_timing] = True
info[_path_install] = modules
model = get_model(info)

[prior] *WARNING* External prior 'SZ' loaded. Mind that it might not be normalized!
[planck_2015_lowl] Importing clik from /home/jesus/scratch/modules/code/planck_2015
[planck_2015_lowTEB] Importing clik from /home/jesus/scratch/modules/code/planck_2015
[planck_2015_plikHM_TT] Importing clik from /home/jesus/scratch/modules/code/planck_2015
[planck_2015_plikHM_TTTEEE] Importing clik from /home/jesus/scratch/modules/code/planck_2015
[planck_2015_plikHM_TT_unbinned] Importing clik from /home/jesus/scratch/modules/code/planck_2015
[planck_2015_plikHM_TTTEEE_unbinned] Importing clik from /home/jesus/scratch/modules/code/planck_2015
[planck_2015_lensing] Importing clik from /home/jesus/scratch/modules/code/planck_2015
[planck_2015_lensing_cmblikes] Reading data from /home/jesus/scratch/modules/data/planck_supp_data_and_covmats/lensing/smica_g30_ftl_full_pp.dataset
[bicep_keck_2014] Reading data from /home/jesus/scratch/modules/data/bicep_keck_2014/BK14_cosmomc/data/BK14/BK14_dust.dataset
[s

In [3]:
params_lowTEB_highTTTEEE.pop("H0", None)

for i in range(n):
    point = dict(zip(model.parameterization.sampled_params(), model.prior.sample(ignore_external=True)[0]))
    point.update(params_lowTEB_highTTTEEE)
    model.loglike(point, cached=False)

In [4]:
from pandas import DataFrame
df = DataFrame(columns=["time", "speed", "old speed"], index=likelihoods)
for name in [_theory] + list(model.likelihood):
    like = model.likelihood[name]
    df.at[name, "time"] = like.time_avg
    df.at[name, "speed"] = 1/like.time_avg
    df.at[name, "old speed"] = like.speed
    df.at[name, "factor"] = like.speed*like.time_avg
print(df)

                                           time    speed old speed    factor
planck_2015_lowl                    0.000251658  3973.65      2900  0.729808
planck_2015_lowTEB                     0.245518  4.07302         4  0.982072
planck_2015_plikHM_TT                 0.0107683  92.8655        60  0.646096
planck_2015_plikHM_TTTEEE             0.0380948  26.2503        25  0.952370
planck_2015_plikHM_TT_unbinned          0.12392  8.06975        10  1.239196
planck_2015_plikHM_TTTEEE_unbinned     0.587045  1.70345         2  1.174090
planck_2015_lensing                   0.0012351   809.65       800  0.988081
planck_2015_lensing_cmblikes        0.000916348  1091.29       700  0.641443
bicep_keck_2014                       0.0109873  91.0144       100  1.098727
H0_riess2018a                       0.000242432  4124.86      6500  1.575810
H0_riess2018b                       0.000186495  5362.09      6500  1.212215
sdss_dr12_consensus_bao             0.000636946  1569.99      2500  1.592365

In [7]:
from pandas import DataFrame
df = DataFrame(columns=["time", "speed", "old speed"], index=likelihoods)
for name in [_theory] + list(model.likelihood):
    like = model.likelihood[name]
    df.at[name, "time"] = like.time_avg
    df.at[name, "speed"] = 1/like.time_avg
    df.at[name, "old speed"] = like.speed
    df.at[name, "factor"] = like.speed*like.time_avg
print(df)

                                           time     speed old speed    factor
planck_2015_lowl                    0.000328891   3040.52      2900  0.953785
planck_2015_lowTEB                     0.243641   4.10439         4  0.974565
planck_2015_plikHM_TT                 0.0118594   84.3216        60  0.711561
planck_2015_plikHM_TTTEEE              0.038316   26.0987        25  0.957901
planck_2015_plikHM_TT_unbinned         0.123394   8.10415        10  1.233935
planck_2015_plikHM_TTTEEE_unbinned     0.587823   1.70119         2  1.175646
planck_2015_lensing                  0.00120521   829.733       800  0.964166
planck_2015_lensing_cmblikes        0.000872457   1146.19       700  0.610720
bicep_keck_2014                       0.0115133   86.8561       100  1.151330
H0_riess2018a                       0.000269573   3709.57      6500  1.752223
H0_riess2018b                       0.000215471      4641      6500  1.400560
sdss_dr12_consensus_bao             0.000690948   1447.29      2